# WIP
# Test significance of S1, S2, D1, D2
The idea is to follow the procedure of the paper "Assessing harmfulness and vulnerability in global bipartite networks of
terrorist-target relationships" A. Spelta et al.

In [ ]:
import sys

sys.path.insert(0, "../")

import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scripts.hid_haus import hid_hous
from scripts.get_graph import get_graph

## Get Adjacency matrix and graph

In [ ]:
# Get the adjacency matrix from the data
adjacency_matrix = pd.read_csv("../../dataset/adjacency_matrix.csv").values

# Remove the last column of the adjacency matrix
adjacency_matrix = adjacency_matrix[:, :-1]
graph = get_graph(adjacency_matrix)

## Null Model Construction (Level-1)


In [ ]:
# Define a function to generate random graphs while preserving degrees
def generate_null_model(A):
    G_null = nx.configuration_model(np.sum(A, axis=1))
    return nx.Graph(G_null)


# Create an ensemble of random networks
num_null_models = 100  # You can adjust this number
null_models = [generate_null_model(adjacency_matrix) for _ in range(num_null_models)]

## WIP
## Null Model Construction (Level-2):


In [ ]:
# theta_U and theta_u are the unknown parameter vectors
def loss_function(theta_U, theta_u):
    # Calculate the log-likelihood based on the provided formula
    log_likelihood = (
        np.sum(np.log(theta_U))
        + np.sum(np.log(theta_u))
        + np.log(1 - np.dot(theta_U, theta_u))
    )

    # Minimize the negative log-likelihood
    return -log_likelihood


initial_guess_U = 0
initial_guess_u = 0
result = minimize(
    loss_function, x0=(initial_guess_U, initial_guess_u), method="L-BFGS-B"
)

# Extract optimal parameters
optimal_theta_U, optimal_theta_u = result.x

## Statistical Validation:


In [ ]:
# Compute the same measures for each null model
null_model_measures = [hid_hous(null_model, 3) for null_model in null_models]


# Calculate Z-scores for statistical significance
def calculate_z_score(real_value, null_model_values):
    mean_null = np.mean(null_model_values)
    std_null = np.std(null_model_values)
    z_score = (real_value - mean_null) / std_null
    return z_score

[real_kd, real_ks] = hid_hous(adjacency_matrix, 3)

# Compute Z-scores for your measures
z_score_kd = calculate_z_score(real_kd, [kd for kd, _ in null_model_measures])
z_score_ks = calculate_z_score(real_ks, [ks for _, ks in null_model_measures])
